# Pair Programming Limpieza I

En este ejercicio trabajaremos con un csv nuevo. En concreto trabajaremos con datos de banca/marketing. Este conjunto de datos contiene información sobre campañas de marketing bancario, incluyendo detalles como la edad, el trabajo, el estado civil, etc. En concreto estos datos son de una campaña de marketing para la venta de un producto o servicio, la columna ("y") parece indicar si el cliente ha aceptado la oferta o no.
Tenemos distintos tipos de variables en el dataset, veamos que significan:
Datos del cliente bancario:

- age: (numérico)
- job : tipo de trabajo (categórico: "administrativo", "obrero", "empresario", "empleada del hogar", "directivo", "jubilado", "autónomo", "servicios", "estudiante", "técnico", "desempleado")
- marital : estado civil (categórico: "divorciado", "casado", "soltero". Nota: "divorciado" significa divorciado o viudo)
- education (categórica: "básica.4a", "básica.6a", "básica.9a", "secundaria", "analfabeto", "curso.profesional", "título.universitario")
- default: ¿tiene crédito en mora? (categórico: O: "no", 1: "sí")
- housing: ¿tiene préstamo para vivienda? (categórico: 0: "no", 1: "sí")
- loan: ¿tiene préstamo personal? (categórico: 0: "no", 1: "sí")

Datos relacionados con el último contacto de la campaña actual:

- contact: tipo de comunicación del contacto (categórico: "móvil", "teléfono")
- month_day_week: lista con el año del último contacto (categórico: "ene", "feb", "mar", ..., "nov", "dic") y el día de la semana del último contacto (categorical: "mon", "tue", "wed", "thu", "fri")
- duration: duración del último contacto, en segundos (numérico). Nota importante: este atributo afecta en gran medida al objetivo de salida (por ejemplo, si duration=0 entonces y="no"). Sin embargo, la duración no se conoce antes de realizar una llamada. Además, es obvio que y se conoce una vez finalizada la llamada. Por lo tanto, esta entrada sólo debería incluirse a efectos de evaluación comparativa y debería descartarse si la intención es tener un modelo predictivo realista.

Otras variables:
- campaign: número de contactos realizados durante esta campaña y para este cliente (numérico, incluye el último contacto)
- pdays: número de días transcurridos desde que el cliente fue contactado por última vez en una campaña anterior (numérico; 999 significa que el cliente no fue contactado previamente)
- previous: número de contactos realizados antes de esta campaña y para este cliente (numérico)
- poutcome: resultado de la campaña de marketing anterior (categórico: "fracaso", "inexistente", "éxito")
- y: ¿ha suscrito el cliente un depósito a plazo? (binario: "sí", "no")

Variables de contexto social y económico:
- emp.var.rate: tasa de variación del empleo - indicador trimestral (numérico)
- cons.price.idx: índice de precios al consumidor - indicador mensual (numérico)
- cons.conf.idx: índice de confianza del consumidor - indicador mensual (numérico)
- euribor3`m: tipo euribor a 3 meses - indicador diario (numérico)
- nr.employed: número de empleados - indicador trimestral (numérico)

## Hipótesis

1. La edad, el trabajo, el estado civil, la educación, la situación de deuda y la forma de contacto pueden influir en la probabilidad de que un cliente acepte la oferta.

2. El número de veces que se ha contactado a un cliente en el pasado (campo campaign), el número de días que han pasado desde el último contacto (campo pdays), y el resultado de la campaña anterior (campo poutcome) pueden afectar la respuesta del cliente a una nueva oferta.

3. Las variables económicas (tales como el índice de precios al consumidor (cons.price.idx), la tasa de variación del empleo (emp.var.rate), etc.) pueden influir en la probabilidad de que un cliente acepte la oferta.
4. Los clientes que ya tienen una hipoteca (housing) o un préstamo (loan) pueden ser menos propensos a aceptar una nueva oferta, ya que podrían estar limitados financieramente.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_bank = pd.read_csv("datos/bank-additional-full.csv", index_col= 0)
df_bank

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,month_day_week
0,56,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
1,57,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
2,37,services,MARRIED,high.school,0.0,1.0,0.0,telephone,226,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
3,40,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,151,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
4,56,services,MARRIED,high.school,0.0,0.0,1.0,telephone,307,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15280,36,admin.,MARRIED,university.degree,NaN,0.0,0.0,cellular,674,3,999,0,NONEXISTENT,1.4,"93,918","-42,7","4,958","5228,1",no,"['jul', 'thu']"
27570,37,unemployed,SINGLE,university.degree,0.0,0.0,1.0,cellular,104,2,999,0,NONEXISTENT,-0.1,"93,2","-42,0","4,021","5195,8",no,"['nov', 'fri']"
17746,47,services,DIVORCED,professional.course,0.0,0.0,0.0,cellular,318,1,999,0,NONEXISTENT,1.4,"93,918","-42,7","4,961","5228,1",no,"['jul', 'tue']"
30574,42,admin.,MARRIED,university.degree,0.0,1.0,0.0,cellular,20,5,999,0,NONEXISTENT,-1.8,"92,893","-46,2","1,354","5099,1",no,"['may', 'mon']"


Entre las tareas que tendréis que hacer hoy:
- Explora el dataset con los métodos que hemos ido aprendiendo hasta ahora para familiarizarnos con los datos.


In [3]:
display(df_bank.head(2))
display(df_bank.tail(2))
display(df_bank.sample(2))

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,month_day_week
0,56,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
1,57,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"


,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,month_day_week
30574,42,admin.,MARRIED,university.degree,0.0,1.0,0.0,cellular,20,5,999,0,NONEXISTENT,-1.8,"92,893","-46,2","1,354","5099,1",no,"['may', 'mon']"
29612,35,services,MARRIED,basic.9y,NaN,1.0,0.0,cellular,201,3,999,0,NONEXISTENT,-1.8,"93,075","-47,1","1,405","5099,1",no,"['apr', 'mon']"


,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,month_day_week
12557,52,entrepreneur,MARRIED,university.degree,0.0,1.0,0.0,cellular,97,3,999,0,NONEXISTENT,1.4,"93,918","-42,7","4,96","5228,1",no,"['jul', 'mon']"
28999,35,blue-collar,MARRIED,NaN,0.0,0.0,0.0,cellular,172,1,999,0,NONEXISTENT,-1.8,"93,075","-47,1","1,405","5099,1",no,"['apr', 'fri']"


In [4]:
df_bank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43170 entries, 0 to 29612
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             43170 non-null  int64  
 1   job             42823 non-null  object 
 2   marital         43085 non-null  object 
 3   education       41355 non-null  object 
 4   default         34154 non-null  float64
 5   housing         42139 non-null  float64
 6   loan            42139 non-null  float64
 7   contact         43170 non-null  object 
 8   duration        43170 non-null  int64  
 9   campaign        43170 non-null  int64  
 10  pdays           43170 non-null  int64  
 11  previous        43170 non-null  int64  
 12  poutcome        43170 non-null  object 
 13  emp.var.rate    43170 non-null  float64
 14  cons.price.idx  43170 non-null  object 
 15  cons.conf.idx   43170 non-null  object 
 16  euribor3m       43170 non-null  object 
 17  nr.employed     43170 non-null 

In [5]:
df_bank.describe().T

,count,mean,std,min,25%,50%,75%,max
age,43170.0,40.014918,10.407327,17.0,32.0,38.0,47.0,98.0
default,34154.0,0.000088,0.009372,0.0,0.0,0.0,0.0,1.0
housing,42139.0,0.535869,0.498718,0.0,0.0,1.0,1.0,1.0
loan,42139.0,0.155652,0.362529,0.0,0.0,0.0,0.0,1.0
duration,43170.0,257.883739,259.393738,0.0,102.0,179.0,319.0,4918.0
campaign,43170.0,2.564675,2.768538,1.0,1.0,2.0,3.0,56.0
pdays,43170.0,962.452282,186.962379,0.0,999.0,999.0,999.0,999.0
previous,43170.0,0.173639,0.496750,0.0,0.0,0.0,0.0,7.0
emp.var.rate,43170.0,0.078177,1.572217,-3.4,-1.8,1.1,1.4,1.4


In [6]:
df_bank.describe(include = "object").T

,count,unique,top,freq
job,42823,11,admin.,10918
marital,43085,3,MARRIED,26105
education,41355,7,university.degree,12766
contact,43170,2,cellular,27438
poutcome,43170,3,NONEXISTENT,37261
cons.price.idx,43170,26,"93,994",8117
cons.conf.idx,43170,26,"-36,4",8117
euribor3m,43170,316,"4,857",3002
nr.employed,43170,11,"5228,1",16981
y,43170,2,no,38308



- Cambia el nombre de las columnas para que no tengan puntos en el nombre. En concreto, reemplazad los "." por comas.


- lo cambiamos por "_" porque la coma nos da toc.

In [7]:
nuevas_columnas = {col: col.replace(".", "_").lower() for col in df_bank.columns}
df_bank.rename(columns = nuevas_columnas, inplace = True)
df_bank.head(1)

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,month_day_week
0,56,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"


- ¿Hay valores duplicados en nuestro dataframe? En caso de que los haya, eliminándlos.


In [8]:
df_bank.duplicated().sum()

1994

In [9]:
df_bank.drop_duplicates(inplace = True)
df_bank

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,month_day_week
0,56,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
1,57,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
2,37,services,MARRIED,high.school,0.0,1.0,0.0,telephone,226,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
3,40,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,151,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
4,56,services,MARRIED,high.school,0.0,0.0,1.0,telephone,307,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,MARRIED,professional.course,0.0,1.0,0.0,cellular,334,1,999,0,NONEXISTENT,-1.1,"94,767","-50,8","1,028","4963,6",yes,"['nov', 'fri']"
41184,46,blue-collar,MARRIED,professional.course,0.0,0.0,0.0,cellular,383,1,999,0,NONEXISTENT,-1.1,"94,767","-50,8","1,028","4963,6",no,"['nov', 'fri']"
41185,56,retired,MARRIED,university.degree,0.0,1.0,0.0,cellular,189,2,999,0,NONEXISTENT,-1.1,"94,767","-50,8","1,028","4963,6",no,"['nov', 'fri']"
41186,44,technician,MARRIED,professional.course,0.0,0.0,0.0,cellular,442,1,999,0,NONEXISTENT,-1.1,"94,767","-50,8","1,028","4963,6",yes,"['nov', 'fri']"


In [10]:
df_bank.duplicated().sum()

0

- Como hemos visto, algunas columnas no tienen el tipo de datos que deberían. Cambiad el tipo de dato para las columnas de cons.price.idx, cons.conf.idx, euribor3m, nr.employed.


In [11]:
df_bank[["cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"]] = df_bank[["cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"]].replace(',', ".", regex=True)
df_bank.head(2)

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,month_day_week
0,56,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']"
1,57,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']"


In [12]:
df_bank = df_bank.astype({"cons_price_idx": "float64", "cons_conf_idx": "float64", "euribor3m": "float64", "nr_employed" : "float64"}, copy=True, errors='raise')
df_bank.head(2)

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,month_day_week
0,56,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']"
1,57,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']"


In [13]:
df_bank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41176 entries, 0 to 41187
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41176 non-null  int64  
 1   job             40846 non-null  object 
 2   marital         41096 non-null  object 
 3   education       39446 non-null  object 
 4   default         32580 non-null  float64
 5   housing         40186 non-null  float64
 6   loan            40186 non-null  float64
 7   contact         41176 non-null  object 
 8   duration        41176 non-null  int64  
 9   campaign        41176 non-null  int64  
 10  pdays           41176 non-null  int64  
 11  previous        41176 non-null  int64  
 12  poutcome        41176 non-null  object 
 13  emp_var_rate    41176 non-null  float64
 14  cons_price_idx  41176 non-null  float64
 15  cons_conf_idx   41176 non-null  float64
 16  euribor3m       41176 non-null  float64
 17  nr_employed     41176 non-null 

- En las columnas de marital y month, poner todos los valores en minúsculas.
💡 Pista Podréis usar una función o una lambda 💡


In [19]:
for col in ["marital", "month_day_week"]:
    df_bank[col] = df_bank[col].apply(lambda cesar: cesar.lower() if type(cesar) == str else cesar)

df_bank.head(2)

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,month_day_week
0,56,housemaid,married,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']"
1,57,services,married,high.school,NaN,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']"


- Para no olvidarnos de la visualización. A través de la visualización, podemos detectar errores, valores atípicos, valores faltantes y otros problemas comunes en los datos que de otra manera podrían ser difíciles de identificar. Para esto vamos a hacer una serie de gráficas que deberán ser incluidas en un subplot. Estas gráficas tienen que ser:

        - Una gráfica donde podamos ver si el tipo de trabajo afecta a que un cliente acepte la oferta.


        - Una gráfica donde podamos ver si el hecho de que un cliente tenga una hipoteca afecta a que acepte la oferta.


        - Una gráfica donde podamos ver si el hecho de que un cliente tenga un prestamo afecta a que acepte la oferta.


        - Una gráfica donde podamos ver si hay una relación entre el numero de días desde el último contacto afecta a que un cliente acepte la oferta


        - Una gráfica donde podamos ver si hay relación entre el resultado de la campaña anterior y si el cliente afecta a que el cliente acepte la oferta.


- Guardamos el csv para seguir trabajando en el siguiente ejercicio de pair de limpieza.